In [20]:
# Initialize GAPID environment
from helpers import gapid_env
from importlib import reload
reload(gapid_env)
import os

pathdir = globals()['_dh'][0]
env = gapid_env.gapid_env(os.path.normpath(os.path.join(pathdir, "..", "build", "Debug")))


Status: GAPID environment initialized
Status:     Gapir: d:\src\gapid\build\Debug\gapir.exe
Status:     Printer: d:\src\gapid\build\Debug\printer.exe


In [2]:
# Select and load trace file
from helpers import trace as gapid_trace
from importlib import reload
reload(gapid_trace)

from ipyfilechooser import FileChooser


fc = FileChooser()
if env.current_trace:
    fc.default_path = os.path.dirname(env.current_trace)
    fc.default_filename = os.path.basename(env.current_trace)

fc.filter_pattern = "*.trace"
fc.title = "<b>Select Trace</b>"
trace = None
def change_file(chooser):
    global trace
    current_trace = os.path.join(chooser.selected_path, chooser.selected_filename)
    trace =  gapid_trace.load_trace(env, current_trace)
fc.register_callback(change_file)
display(fc)

FileChooser(path='D:\src\gapid\Python', filename='', title='<b>Select Trace</b>', show_hidden=False, select_de…

Status: Loading trace D:\src\file.trace
Status: Loaded trace D:\src\file.trace


In [23]:
# Report stats about the trace
import json
from importlib import reload
reload(gapid_trace)

cmd_stats = trace.get_stats()
print(json.dumps(cmd_stats, indent="    "))


{
    "Total Commands": 238839,
    "Frame Count": 10,
    "Loading time": "21.0s",
    "MEC Commands": 193159,
    "Application Commands": 45680,
    "Average Commands Per Frame": 4568.0
}


In [5]:
# Count the draw commands actually submitted
from collections import defaultdict

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & gapid_trace.MID_EXECUTION) == 0)]

submitted_draws = []
submitted_dispatches = []
for x in queue_submits:
    executed_cbs = trace.executed_commands[x]
    for y in executed_cbs:
        submitted_draws.extend([x for x in y if "Draw" in trace.commands[x]["name"]])
        submitted_dispatches.extend([x for x in y if "Dispatch" in trace.commands[x]["name"]])
l = defaultdict(list)
for x in submitted_draws:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Draws: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

l = defaultdict(list)
for x in submitted_dispatches:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Dispatches: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

Total Draws: 8335
  vkCmdDrawIndexed: 6685
  vkCmdDrawIndexedIndirect: 110
  vkCmdDraw: 1540
Total Dispatches: 8335
  vkCmdDispatch: 360


In [86]:
from helpers import replay
from importlib import reload
reload(replay)

opts = replay.replay_options(trace)
opts.use_callback_swapchain()

replay.replay(env, opts)

Status: Trying to run d:\src\gapid\build\Debug\gapir.exe --socket localhost --port 55554 D:\src\file.trace
{'Message': 'Info', 'Time': Decimal('4.04482'), 'Content': 'Initializing time:: 2.037782'}
{'Message': 'Info', 'Time': Decimal('10.1829'), 'Content': 'Mec Time:: 5.3990197'}
{'Message': 'Info', 'Time': Decimal('10.183'), 'Content': 'Run time:: 0.738911'}
